In [ ]:
import numpy as np
import random
import pandas as pd
import gurobipy as gp

# Question 1: Integer Program to Maximize Mean Return of Portfolio

In [ ]:
url = "https://raw.githubusercontent.com/jake-p-johnson/Optimization2/main/HW2/stocks2019.csv"
stocks2019 = pd.read_csv(url)
url2 = "https://raw.githubusercontent.com/jake-p-johnson/Optimization2/main/HW2/stocks2020-1.csv"
stocks2020 = pd.read_csv(url2)

In [ ]:
nstocks = stocks2019.shape[1]-2
ndays = stocks2019.shape[0]
ret2019 = stocks2019.iloc[1:ndays,2:nstocks+2].values - stocks2019.iloc[0:ndays-1,2:nstocks+2].values
ret2019 = np.divide(ret2019,stocks2019.iloc[0:ndays-1,2:nstocks+2].values)
ndays = ret2019.shape[0]

In [ ]:
ndays2020 = stocks2020.shape[0]
ret2020 = stocks2020.iloc[1:ndays2020,2:nstocks+2].values - stocks2020.iloc[0:ndays2020-1,2:nstocks+2].values
ret2020 = np.divide(ret2020,stocks2020.iloc[0:ndays2020-1,2:nstocks+2].values)
ndays2020 = ret2020.shape[0]
meanvec = np.mean(ret2019,0)

In [ ]:
c = np.concatenate((meanvec,[0]*ndays))
A = np.zeros((2+ndays,nstocks+ndays))
b = np.zeros((2+ndays,1))

In [ ]:
A[0,0:nstocks]=1
b[0] = 1

#no more than 100 days negative
A[1,nstocks:nstocks+ndays] = 1
b[1] = 100

# enforce big M constraint
M = 10
for day in range(1,ndays+1):
    A[1+day,0:nstocks] = ret2019[day-1,0:nstocks]
    A[1+day,nstocks+day-1] = M
    b[1+day] = 0

sense = np.concatenate((['='],['<'],['>']*ndays))

In [ ]:
m = gp.Model()
mX = m.addMVar(len(c),vtype = np.concatenate((['C']*nstocks,['B']*ndays)))
m.setMObjective(None, c, 0, sense = gp.GRB.MAXIMIZE)
m.addMConstrs(A, mX, sense, b)
m.Params.OutputFlag = 1

#Solve the IP using gurobi 
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 252 rows, 350 columns and 25504 nonzeros
Model fingerprint: 0x5c0d199b
Variable types: 100 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [2e-05, 1e+01]
  Objective range  [7e-05, 4e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.09s
Presolved: 251 rows, 349 columns, 25402 nonzeros
Variable types: 100 continuous, 249 integer (249 binary)

Root relaxation: objective 4.097099e-03, 112 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00410    0   18          -    0.00410      -     -    0s
     0     0    0.00410    0   25          -    0.00410      -     -    0s
 

In [ ]:
num_neg = 0
for day in range(1,ndays2020+1):
    port_ret = sum(np.multiply(mX.x[0:nstocks],ret2020[day-1,0:nstocks]))
    if(port_ret < 0):
        num_neg = num_neg + 1
        
print(num_neg)

115


# Question 2: Newsvendor Problem

In [ ]:
urld = "https://raw.githubusercontent.com/jake-p-johnson/Optimization2/main/HW2/hw2_demand.csv"
demand = pd.read_csv(urld)
demand.head()

,X1,X2,X3,X4,X5
0,19,1358,95,65,4994
1,31,734,153,106,1711
2,45,732,198,348,205
3,1,57,64,89,278
4,180,316,277,176,46


In [ ]:
npap = demand.shape[1]
nd = demand.shape[0]
costs = np.array([0.5,0.5,0.25,0.75,0.25])
prices = np.array([1.25,1.2,0.5,1.5,0.6])

In [ ]:
# decision variables (q1..q5, hij)
obj = np.concatenate(([0]*npap, [1/nd]*nd*npap))
lb = np.concatenate(([0]*npap, [-np.inf]*nd*npap)) #quantity non negative but profit on a day can be negative
A = np.zeros((2*nd*npap, nd*npap+npap))
b = np.zeros((2*nd*npap,1))
sense = np.array((['<']*2*nd*npap))

In [ ]:
for news in range(1, npap+1):
  for day in range(1, nd+1):
    cd = day + (news-1)*nd
    A[2*cd-1-1,news-1] = costs[news-1]
    A[2*cd-1-1,cd+npap-1] = 1
    b[2*cd-1-1] = prices[news-1]*demand.iloc[day-1,news-1]
    A[2*cd-1,news-1] = costs[news-1] - prices[news-1]
    A[2*cd-1,cd+npap-1] = 1

In [ ]:
m = gp.Model()
mX = m.addMVar(len(obj),vtype = ['C']*len(obj),lb = lb)
m.setMObjective(None, obj, 0, sense = gp.GRB.MAXIMIZE)
m.addMConstrs(A, mX, sense, b)
m.Params.OutputFlag = 1

m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 500 rows, 255 columns and 1000 nonzeros
Model fingerprint: 0x679bc82e
Coefficient statistics:
  Matrix range     [2e-01, 1e+00]
  Objective range  [2e-02, 2e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+03]
Presolve removed 250 rows and 0 columns
Presolve time: 0.01s
Presolved: 250 rows, 255 columns, 500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4081840e+03   7.040920e+04   0.000000e+00      0s
     115    3.3698900e+02   0.000000e+00   0.000000e+00      0s

Solved in 115 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.369890000e+02


In [ ]:
print(mX.x[0:npap]) # how many we should produce
print(m.objval) # expectation of profits given that many produced

[ 76. 462. 205. 123. 422.]
336.9890000000003


# Question 3: Simulation of Saving For Retirement

In [ ]:
nsim = 10000
def_vec = np.array([False]*nsim)
port_val_vec = np.zeros(nsim)
years_working = 3
for sim in range(nsim):
    value = 2000000
    salary = 400000
    for year in range(1,36):
      port_return = np.random.normal(size = 1, loc = 0.07, scale = 0.1)
      value = value*(1+port_return)
      value = value - 150000
      if (year <= years_working):
        value = value + salary
        salary = salary * (1.025)
    
    if value <= 0:
      def_vec[sim] = True
    port_val_vec[sim] = value

np.mean(def_vec)

0.0528

In [ ]:
port_val_vec

array([2.23458177e+16, 2.21225000e+16, 2.14045524e+16, ...,
       2.30569709e+16, 2.31887059e+16, 2.26089803e+16])

# Question 4: Manufacturing Simulation  
Process1 - normrandom(mean=10, sd=3) costs 10 produce 100  
Process2 - normrandom(mean=5, sd=2) costs 5 produce 200  
performance < 7 low performance sell 8  
performance > 7 high performance sell 15

In [ ]:
nsim = 10000
profit_vec = [0] * nsim
for sim in range(1, nsim+1):
  chipsAperf = np.random.normal(size = 100, loc = 10, scale = 3) # get performance of all chips manufactured using first method
  chipsBperf = np.random.normal(size = 200, loc = 5, scale = 2)
  #process one
  profitA = [0]*100
  for i in range(100):
    if(chipsAperf[i] >= 7):
      profitA[i] = 15-10
    else:
      profitA[i] = 8-10
  
  #process two
  profitB = [0]*200
  for i in range(200):
    if (chipsBperf[i] >= 7):
      profitB[i] = 15-5
    else:
      profitB[i] = 8-5
  
  profit_vec[sim-1] = sum(profitA) + sum(profitB)

np.quantile(profit_vec,np.array([0.25,0.975]))

array([1184., 1296.])

In [ ]:
np.std(profit_vec)

44.58639896997738